In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.0 MB/s eta 0:00:00


In [1]:
# ===================================================================
#  Library
# ===================================================================
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import math
import time


from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_absolute_percentage_error
from tqdm.auto import tqdm

import warnings
warnings.simplefilter("ignore")

import unicodedata
import lightgbm as lgb

import optuna
import tensorflow as tf
from sklearn.metrics import mean_absolute_percentage_error
from tensorflow.keras.layers import Dense, PReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [2]:
# ===================================================================
#  CFG
# ===================================================================
class CFG:
    seed = 42
    n_trials = 3000
    num_pred = 13 #予測の数を指定、今回はexp38と39の2つ
    hidden_size = 16
    dropout = 0.2
    save_dir = '/content/drive/MyDrive/Colab Notebooks/signate2023/stacking/exp45/preds/'

In [3]:
# ===================================================================
#  Utils
# ===================================================================
def get_score(y_true, y_pred):
    """get MAPE score"""
    score = mean_absolute_percentage_error(y_true, y_pred)
    return score * 100

In [4]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/train.csv')

In [5]:
df_1 = pd.read_csv(CFG.save_dir+"oof_df_exp014.csv", names=['pred']).rename(columns={"pred":"pred_1"})
df_2 = pd.read_csv(CFG.save_dir+"oof_df_exp015.csv", names=['pred']).rename(columns={"pred":"pred_2"})
df_3 = pd.read_csv(CFG.save_dir+"oof_df_exp016.csv", names=['pred']).rename(columns={"pred":"pred_3"})
df_4 = pd.read_csv(CFG.save_dir+"oof_df_exp017.csv", names=['pred']).rename(columns={"pred":"pred_4"})
df_5 = pd.read_csv(CFG.save_dir+"oof_df_exp018.csv", names=['pred']).rename(columns={"pred":"pred_5"})
df_6 = pd.read_csv(CFG.save_dir+"oof_df_exp019.csv", names=['pred']).rename(columns={"pred":"pred_6"})
df_7 = pd.read_csv(CFG.save_dir+"oof_df_exp020.csv", names=['pred']).rename(columns={"pred":"pred_7"})
df_8 = pd.read_csv(CFG.save_dir+"oof_df_exp021.csv", names=['pred']).rename(columns={"pred":"pred_8"})
df_9 = pd.read_csv(CFG.save_dir+"oof_df_exp022.csv", names=['pred']).rename(columns={"pred":"pred_9"})
df_10 = pd.read_csv(CFG.save_dir+"oof_df_exp023.csv", names=['pred']).rename(columns={"pred":"pred_10"})
df_11 = pd.read_csv(CFG.save_dir+"exp38_oof_pred.csv").rename(columns={"oof_pred":"pred_11"})
df_12 = pd.read_csv(CFG.save_dir+"exp39_oof_pred.csv").rename(columns={"oof_pred":"pred_12"})
df_13 = pd.read_csv(CFG.save_dir+"exp40_oof_pred.csv").rename(columns={"oof_pred":"pred_13"})

df = pd.concat([df_train['id'], df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10, df_11, df_12, df_13, df_train['price']], axis=1)


test_1 = pd.read_csv(CFG.save_dir+"exp014.csv", header=None).rename(columns={0:"id", 1:"pred_1"})
test_2 = pd.read_csv(CFG.save_dir+"exp015.csv", header=None).rename(columns={0:"id", 1:"pred_2"})
test_3 = pd.read_csv(CFG.save_dir+"exp016.csv", header=None).rename(columns={0:"id", 1:"pred_3"})
test_4 = pd.read_csv(CFG.save_dir+"exp017.csv", header=None).rename(columns={0:"id", 1:"pred_4"})
test_5 = pd.read_csv(CFG.save_dir+"exp018.csv", header=None).rename(columns={0:"id", 1:"pred_5"})
test_6 = pd.read_csv(CFG.save_dir+"exp019.csv", header=None).rename(columns={0:"id", 1:"pred_6"})
test_7 = pd.read_csv(CFG.save_dir+"exp020.csv", header=None).rename(columns={0:"id", 1:"pred_7"})
test_8 = pd.read_csv(CFG.save_dir+"exp021.csv", header=None).rename(columns={0:"id", 1:"pred_8"})
test_9 = pd.read_csv(CFG.save_dir+"exp022.csv", header=None).rename(columns={0:"id", 1:"pred_9"})
test_10 = pd.read_csv(CFG.save_dir+"exp023.csv", header=None).rename(columns={0:"id", 1:"pred_10"})
test_11 = pd.read_csv(CFG.save_dir+"exp38.csv", header=None).rename(columns={0:"id", 1:"pred_11"})
test_12 = pd.read_csv(CFG.save_dir+"exp39.csv", header=None).rename(columns={0:"id", 1:"pred_12"})
test_13 = pd.read_csv(CFG.save_dir+"exp40.csv", header=None).rename(columns={0:"id", 1:"pred_13"})
test = test_1.merge(test_2, on='id')
test = test.merge(test_3, on='id')
test = test.merge(test_4, on='id')
test = test.merge(test_5, on='id')
test = test.merge(test_6, on='id')
test = test.merge(test_7, on='id')
test = test.merge(test_8, on='id')
test = test.merge(test_9, on='id')
test = test.merge(test_10, on='id')
test = test.merge(test_11, on='id')
test = test.merge(test_12, on='id')
test = test.merge(test_13, on='id')

In [6]:
df

,id,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,pred_10,pred_11,pred_12,pred_13,price
0,0,6881.892301,6517.414858,7476.567746,7052.610590,6809.002174,6465.089852,7714.185588,7010.855590,7184.306903,6364.997182,7118.9443,7627.1987,7084.0015,27587
1,1,3740.634027,3496.012582,3789.907986,3322.844722,3603.935425,3485.623398,3507.497452,3714.057283,3488.353022,3750.770625,3660.8042,3526.6816,3732.6855,4724
2,2,2954.247573,2735.110086,3288.647709,2893.840210,3253.350282,2885.690295,3202.103770,2799.365349,2989.931642,2920.912332,2883.7390,3149.7769,2995.7760,10931
3,3,8430.949224,8337.416095,9033.353098,8081.404050,8187.453186,8477.542430,8546.771462,8388.923127,8119.183310,8413.930266,8650.6270,8943.9030,9243.1750,16553
4,4,3972.418866,4254.790314,4089.549517,4272.046434,4446.949951,4022.828501,4374.413436,4369.679027,4150.709500,3941.880817,4031.1462,4406.5480,4107.9190,5158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,12650.381284,13146.560575,12653.178198,11322.272339,13596.740391,12324.404433,11482.192363,11724.735381,11434.271326,13129.896185,12879.4610,13338.9980,12310.2180,32212
27528,27528,6494.944663,6633.645836,6184.736099,6171.310930,6500.089660,6209.227748,6202.691850,6608.664580,6505.449763,6262.582410,6693.9270,6920.0894,6770.7925,5400
27529,27529,12231.954239,15018.441231,14539.286659,14360.762819,11463.997914,13025.753065,12083.865500,12834.191021,13653.370272,11537.194512,14174.7040,15265.7090,14390.8955,22227
27530,27530,6832.933932,6911.537235,6877.397725,7139.135069,6847.030473,6937.347570,6261.849519,6822.260414,6649.630136,6443.098367,7086.4136,6367.2173,6483.3950,3054


In [7]:
test

,id,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,pred_10,pred_11,pred_12,pred_13
0,27532,9665.240676,9060.741324,8931.822504,9993.239922,9387.607212,9908.462741,10128.928844,9738.111509,10579.697128,10122.133738,9462.117521,9616.369529,9455.621674
1,27533,5140.878681,4974.611255,4485.057773,5533.795617,5044.527731,4849.379307,5257.737182,5354.979862,5545.143501,5494.171219,5431.933504,5579.297671,5502.604659
2,27534,5628.236365,5804.522418,5796.571207,5726.217723,5822.816944,5759.308970,5798.455075,5419.099745,5595.549648,5898.592901,5563.460291,5543.298528,5644.606929
3,27535,18624.817802,20021.162475,18835.327938,22007.952770,18960.034958,19900.729462,19313.421537,18475.892150,19846.987451,20655.747438,19615.535011,19333.748566,19484.368965
4,27536,3625.637392,4375.456011,4220.231775,3687.370209,4082.474807,4105.429409,3789.946321,3915.428485,4189.230892,3982.415899,4165.521012,4132.487296,4262.280629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27532,55064,14710.830942,11584.438744,11947.372568,13550.423259,13646.679093,13303.559398,14155.207461,15621.013652,13879.446250,13562.673696,13342.552114,13667.855920,13413.909141
27533,55065,8852.779860,8068.844513,8619.407623,9115.885754,8196.746981,7947.659290,8410.227937,8646.502454,8689.880353,8737.035801,8482.824014,8544.580408,8316.495493
27534,55066,6288.760858,6135.556787,6508.548628,6240.533655,5993.522277,5874.218495,6160.884478,6478.396730,6060.177626,5956.305561,6235.427898,6279.775623,6330.773011
27535,55067,5168.341070,5594.025322,5106.247195,4975.679965,5220.941190,5072.216882,4967.672649,5048.859101,5397.634268,5314.368811,5198.181210,5218.070387,5204.748284


In [8]:
def mape_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    diff = tf.abs((y_true - y_pred) / tf.clip_by_value(tf.abs(y_true), 1e-9, float("inf")))
    return 100. * tf.reduce_mean(diff)

In [9]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(42)  # ここでシード値を設定

In [10]:
# 交差検証関数の定義
def get_custom_cv(df, n_splits):
    df = df.sort_values(by="price", ignore_index=True)
    df["fold"] = [i for i in range(n_splits)] * (df.shape[0] // n_splits) \
                + [i for i in range(df.shape[0] % n_splits)]
    df = df.sort_values(by="id", ignore_index=True)

    for fold in range(n_splits):
        train_idx = df[df["fold"] != fold].index
        valid_idx = df[df["fold"] == fold].index
        yield train_idx, valid_idx

# CVの設定
n_splits = 8
cv = list(get_custom_cv(df, n_splits))

In [11]:
# 1. データセットの作成
X = df[['pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5', 'pred_6', 'pred_7', 'pred_8', 'pred_9', 'pred_10', 'pred_11', 'pred_12', 'pred_13']].values
y = df['price'].values

In [12]:
#一つ目のモデル

In [13]:
# テストデータの特徴量の取得
test_features = test[['pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5', 'pred_6', 'pred_7', 'pred_8', 'pred_9', 'pred_10', 'pred_11', 'pred_12', 'pred_13']].values

# 各フォールドでのテストデータの予測を格納するリスト
test_preds = []

# OOF (Out Of Fold) predictions
oof_preds = np.zeros_like(y)

for fold, (train_idx, valid_idx) in enumerate(cv):
    print(f"Fold {fold + 1}")

    # データの取得
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]

    # モデルの作成と学習
    model = Sequential([
        Dense(CFG.hidden_size*3, input_dim=CFG.num_pred),
        PReLU(),
        Dense(CFG.hidden_size*2),
        PReLU(),
        Dense(CFG.hidden_size),
        PReLU(),
        Dense(1)
    ])

    model.compile(optimizer=Adam(), loss=mape_loss)

    # EarlyStoppingコールバックの定義
    early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, restore_best_weights=True)

    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=32, verbose=1, callbacks=[early_stopping])

    # OOF predictions
    oof_preds[valid_idx] = model.predict(X_valid).reshape(-1)

    print(f"FINISHI: fold{fold} Score: {mean_absolute_percentage_error(y_valid, oof_preds[valid_idx]):.4f}")

    # テストデータの予測
    test_pred = model.predict(test_features).reshape(-1)
    test_preds.append(test_pred)

# Calculate overall OOF score
oof_score = mean_absolute_percentage_error(y, oof_preds)
print("=" * 50)
print(f"FINISHI: Whole OOF Score: {oof_score:.4f}")


Fold 1
Epoch 1/50
753/753 [==============================] - 9s 4ms/step - loss: 44.8640 - val_loss: 43.3984
Epoch 2/50
753/753 [==============================] - 2s 2ms/step - loss: 44.1460 - val_loss: 43.3883
Epoch 3/50
753/753 [==============================] - 2s 2ms/step - loss: 44.1449 - val_loss: 43.3960
Epoch 4/50
753/753 [==============================] - 2s 2ms/step - loss: 44.1032 - val_loss: 43.6948
Epoch 5/50
753/753 [==============================] - 2s 2ms/step - loss: 44.1432 - val_loss: 43.4734
Epoch 6/50
753/753 [==============================] - 2s 3ms/step - loss: 44.0861 - val_loss: 43.6089
Epoch 7/50
753/753 [==============================] - 3s 4ms/step - loss: 44.1036 - val_loss: 43.3782
Epoch 8/50
753/753 [==============================] - 2s 3ms/step - loss: 44.1068 - val_loss: 43.3800
Epoch 9/50
753/753 [==============================] - 2s 2ms/step - loss: 44.0881 - val_loss: 43.3887
Epoch 10/50
753/753 [==============================] - 2s 2ms/step - loss: 

In [14]:
hidden_size = 8
FINISHI: Whole OOF Score: 0.4395

hidden_size = 16
FINISHI: Whole OOF Score: 0.4399

SyntaxError: ignored

In [ ]:
#二つ目のモデル、こっちはあんまり期待できなさそうかも？

In [ ]:
'''
from keras.models import Sequential
from keras.layers import Dense, PReLU, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

def build_model(cfg):
    model = Sequential([
        Dense(cfg.hidden_size, input_dim=CFG.num_pred),
        BatchNormalization(),
        Dropout(cfg.dropout),
        PReLU(),
        Dense(cfg.hidden_size),
        BatchNormalization(),
        Dropout(cfg.dropout),
        PReLU(),
        Dense(1)
    ])
    return model

# テストデータの特徴量の取得
test_features = test[['pred_1', 'pred_2']].values

# 各フォールドでのテストデータの予測を格納するリスト
test_preds = []

# OOF (Out Of Fold) predictions
oof_preds = np.zeros_like(y)

for fold, (train_idx, valid_idx) in enumerate(cv):
    print(f"Fold {fold + 1}")

    # データの取得
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]

    # モデルの作成
    model = build_model(CFG)

    model.compile(optimizer=Adam(), loss=mape_loss)

    # EarlyStoppingコールバックの定義
    early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, restore_best_weights=True)

    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=32, verbose=1, callbacks=[early_stopping])

    # OOF predictions
    oof_preds[valid_idx] = model.predict(X_valid).reshape(-1)

    print(f"FINISHI: fold{fold} Score: {mean_absolute_percentage_error(y_valid, oof_preds[valid_idx]):.4f}")

    # テストデータの予測
    test_pred = model.predict(test_features).reshape(-1)
    test_preds.append(test_pred)

# Calculate overall OOF score
oof_score = mean_absolute_percentage_error(y, oof_preds)
print("=" * 50)
print(f"FINISHI: Whole OOF Score: {oof_score:.4f}")
'''

In [ ]:
FINISHI: Whole OOF Score: 0.4452

In [ ]:
#これは共通

In [ ]:
# すべてのフォールドの予測の平均を取る
test["pred"] = np.mean(test_preds, axis=0)

test[["id", "pred"]].to_csv('/content/drive/MyDrive/Colab Notebooks/signate2023/exp45_submission_stacking.csv', index=False, header=None)